In this scrript we download the user investment and break it down at stock level. We then calculate the dataframe at sectoral level and finally we calculate the metrics such as HHI and Expected return (with last year's return as proxy)

In [1]:
import pandas as pd

Downloading the data

In [2]:
all_mf_data_path          = './Data/MF_Data/Results/End_product/20240511_updated_eq_mf_data_v2.xlsx'
    

In [3]:
mf_holding_data               = pd.read_excel(all_mf_data_path, sheet_name = "MF Holding")
mf_meta_data                  = pd.read_excel(all_mf_data_path, sheet_name= "MF Meta")

mf_holding_data.head()
#source of this data is screener

,Fund Id,Stock Invested in,Sector,Sector Total,Value(Mn),% of Total Holdings,1Y Highest Holding,1Y Lowest Holding,Quantity
0,MBS831,Infosys Ltd.,Computers - software & consulting,9.12,2756.5,0.0477,4.85% (Jan 2024),1.86% (Mar 2023),16.47 L
1,MBS831,Sun Pharmaceutical Industries Ltd.,Pharmaceuticals,7.54,1974.1,0.0342,3.42% (Feb 2024),2.41% (May 2023),12.51 L
2,MBS831,State Bank Of India,Public sector bank,5.70,1892.7,0.0328,5.92% (Apr 2023),2.94% (Jan 2024),25.30 L
3,MBS831,Ramkrishna Forgings Ltd.,Castings & forgings,3.21,1851.7,0.0321,5.72% (Aug 2023),3.18% (Jan 2024),23.67 L
4,MBS831,Reliance Industries Ltd.,Refineries & marketing,4.36,1811.1,0.0314,3.14% (Feb 2024),2.03% (Sep 2023),6.20 L


In [4]:
mf_meta_data.head()

,Fund Id,Fund Name,MF ACF,MF Type,Exp_ret
0,MBS831,aditya-birla-sun-life-pure-value-fund-direct-p...,Equity,01_value,0.565269
1,MAA939,axis-value-fund-direct-plan-growth,Equity,01_value,0.577739
2,MAG735,bandhan-sterling-value-fund-direct-plan-growth,Equity,01_value,0.421572
3,MBOA011,baroda-bnp-paribas-value-fund-direct-plan-growth,Equity,01_value,0.686291
4,MCA329,canara-robeco-value-fund-direct-plan-growth,Equity,01_value,0.397082


Load sector mapping to get the sectoral data for the user portfolio (after merge)

In [5]:
mapping_stock                = './Data/MF_Data/Results/End_product/stock_sector_mapping.csv'
stock_sec_map                = pd.read_csv(mapping_stock)
stock_sec_map.head()

,Stock,Stock_ID,Sector_ID
0,360 ONE WAM Ltd.,S1,SEC1
1,3M CO.,S2,SEC2
2,3M India Ltd.,S3,SEC3
3,5Paisa Capital Ltd.,S4,SEC4
4,Aarti Drugs Ltd.,S5,SEC5


Merge sectoral data with user data so now we have data at granulatity of Fund/stock. We can aggregate that to get the data at sectors only. That would help us calculate the sectoral HHI 

In [6]:
mf_holding_data           = pd.merge(mf_holding_data, stock_sec_map, left_on = "Stock Invested in", right_on= "Stock", how = "left")
mf_holding_data           = mf_holding_data[['Fund Id','Stock_ID', 'Sector_ID','% of Total Holdings']]
mf_holding_data           = mf_holding_data.rename(columns={ 'Fund Id' : 'FUND_ID',
                                                             'Stock_ID' : 'STOCK_ID',
                                                             'Sector_ID':'SECTOR_ID',
                                                              '% of Total Holdings' :'STOCK_WEIGHT'})
mf_holding_data.head()

,FUND_ID,STOCK_ID,SECTOR_ID,STOCK_WEIGHT
0,MBS831,S560,SEC9,0.0477
1,MBS831,S1049,SEC5,0.0342
2,MBS831,S1034,SEC58,0.0328
3,MBS831,S911,SEC25,0.0321
4,MBS831,S927,SEC61,0.0314


Now we download user investment data. User specifies the fund id and amount he invests in it.

In [7]:
user_selection_path      = './Data/user_selection_v2.csv'
user_selection           = pd.read_csv(user_selection_path)
user_selection.head()

,FUND_SELECTED,AMOUNT
0,MBS831,1000
1,MBOA011,1000
2,MAA939,1000
3,MAG735,1000
4,MCA329,1000


Merge mutual fund meta data that we have loaded earlier (which has expected return data) with user selection to get the expected return of user existing portfolio and save it.

In [8]:
user_ret               = user_selection
user_ret['WEIGHT']     = user_ret['AMOUNT']/(user_ret['AMOUNT'].sum())
user_ret               = pd.merge(user_ret,mf_meta_data[['Fund Id','Exp_ret']], left_on='FUND_SELECTED', right_on='Fund Id', how='left')  
user_ret['W_R']        = user_ret['WEIGHT']*user_ret['Exp_ret']
#user_ret['W_R'].sum()

with open('user_er.txt', 'w') as file:
    file.write(str(user_ret['W_R'].sum()))

Now we will merge user selection with MF holding data to get stock wise weights in the user portfolio. We call it essentially melted portfolio.

In [9]:
user_mf_melted         = pd.merge(mf_holding_data,user_selection, left_on = "FUND_ID", right_on = "FUND_SELECTED", how = "inner")
user_mf_melted['AMOUNT'] = user_mf_melted['AMOUNT']*user_mf_melted['STOCK_WEIGHT']

user_mf_melted.head()

,FUND_ID,STOCK_ID,SECTOR_ID,STOCK_WEIGHT,FUND_SELECTED,AMOUNT,WEIGHT
0,MBS831,S560,SEC9,0.0477,MBS831,47.7,0.025641
1,MBS831,S1049,SEC5,0.0342,MBS831,34.2,0.025641
2,MBS831,S1034,SEC58,0.0328,MBS831,32.8,0.025641
3,MBS831,S911,SEC25,0.0321,MBS831,32.1,0.025641
4,MBS831,S927,SEC61,0.0314,MBS831,31.4,0.025641


Having details at such granularity, we cancalculate sectoral HHI and save it in a file

In [10]:
user_mf_sec       = user_mf_melted.groupby(['FUND_ID','SECTOR_ID'])['AMOUNT'].sum()
user_mf_sec       = user_mf_sec.reset_index() 
user_mf_sec['USER_WEIGHT'] = user_mf_sec['AMOUNT']/(user_mf_sec['AMOUNT'].sum())
user_mf_sec.head()

,FUND_ID,SECTOR_ID,AMOUNT,USER_WEIGHT
0,MAA181,SEC113,25.5,0.000654
1,MAA181,SEC117,61.4,0.001574
2,MAA181,SEC128,1.5,0.000038
3,MAA181,SEC13,14.0,0.000359
4,MAA181,SEC132,17.1,0.000438


In [11]:
user_sec            = user_mf_sec.groupby('SECTOR_ID')['USER_WEIGHT'].sum()
user_sec            = user_sec.reset_index()
user_sec['TOTAL_WEIGHT'] = user_sec['USER_WEIGHT']*100
user_sec['SQ_WEIGHT'] = user_sec['TOTAL_WEIGHT']*user_sec['TOTAL_WEIGHT']
#user_sec['SQ_WEIGHT'].sum()

# Save to a text file
with open('target_hhi.txt', 'w') as file:
    file.write(str(user_sec['SQ_WEIGHT'].sum()))


We come back to melted portfolio; aggregate at stock level and then save it in csv

In [12]:
user_mf_melted         = user_mf_melted.groupby(['STOCK_ID','SECTOR_ID'])['AMOUNT'].sum()
user_mf_melted         = user_mf_melted.reset_index()
user_mf_melted['USER_WEIGHT'] =   user_mf_melted['AMOUNT']/(user_mf_melted['AMOUNT'].sum())
user_mf_melted         = user_mf_melted.sort_values(by= 'SECTOR_ID')
user_mf_melted.head()

,STOCK_ID,SECTOR_ID,AMOUNT,USER_WEIGHT
205,S282,SEC1,36.0,0.000923
550,S894,SEC100,22.8,0.000584
128,S1241,SEC101,5.8,0.000149
38,S1068,SEC102,1.5,0.000038
225,S317,SEC102,95.0,0.002435


In [13]:
user_mf_melted.to_csv('./Data/USER_MF_MELTED_PF_v2.csv', index=False)